In [35]:
import pandas as pd

from collections import Counter
from calibrated_marked_words import calibrated_marked_words as get_marked_words_occ
from og_marked_words import marked_words as get_marked_words_og


In [36]:
df_inferred_gender = pd.read_csv('data/gpt-4o-mini/generated_personas_occupation_inferred_gender_gpt-4o-mini-2024-07-18_100_12-03-2024, 15:29:10.csv')

df_gender = pd.read_csv('data/gpt-4o-mini/generated_personas_occupation_from_winogender_gender_gpt-4o-mini-2024-07-18_100_11-11-2024, 12:02:49.csv')

In [37]:
swe_inferred_gender = df_inferred_gender[df_inferred_gender['occupation'] == 'software engineer']

In [38]:
def pprint(dic, computation = False):
    full_list = []
    if len(dic) == 2:
        computation = dic[1]
        dic = dic[0]
    else:
        computation = None
    for word in sorted(dic,key=lambda x: x[1],reverse=True):
        full_list.append(word[0])
    if computation:
        return full_list, computation
    else:
        return full_list

In [39]:
names = pd.read_csv('names_with_dem.csv')

first_names = list()
last_names = list()

for name in names['First Name'].unique():
    if type(name) == str:
        first_names.append(name.lower())
for name in names['Last Name'].unique():
    if type(name) == str:
        last_names.append(name.lower())

lower_names = set(first_names + last_names)

In [40]:
def compute_marked_word_og_and_calibrated(df, og_computation=False, inferred_gender=False, return_computation = True):

    if inferred_gender:
        gender_col = 'inferred_gender'
    else:
        gender_col = 'gender'

    dv3_mw = dict()
    dv3_mw_counts = dict()
    dv3_mw_names = dict()
    dv3_mw_occ = dict()
    dv3_mw_occ_counts = dict()
    dv3_mw_occ_names = dict()
    dv3_mw_by_occ = dict()
    dv3_mw_by_occ_counts = dict()
    dv3_mw_by_occ_names = dict()
    remove_names = False
    for occupation in ['software engineer']:
        dv3_mw_by_occ[occupation] = dict()
        dv3_mw_by_occ_counts[occupation] = dict()
        dv3_mw_by_occ_names[occupation] = dict()
        
        for race in df[gender_col].unique():
            if og_computation:
                outs = pprint(get_marked_words_og(df, [race], [gender_col],['M'], occupation)) # might need to remove occupation
            else:
                outs = pprint(get_marked_words_occ(df, [race], [gender_col],['M'], occupation))
            new_outs = list()
            curr_names = list()
            for word in outs:
                if remove_names and (word in lower_names or word[:-1] in lower_names):
                    curr_names.append(word)
                else:
                    new_outs.append(word)
            if race in dv3_mw:
                dv3_mw[race].append([new_outs])
                dv3_mw_names[race].append([[curr_names]])
                if race in dv3_mw_by_occ[occupation]:
                    dv3_mw_by_occ[occupation][race].append(new_outs)
                    dv3_mw_by_occ_names[occupation][race].append(curr_names)
                else:
                    dv3_mw_by_occ[occupation][race] = new_outs
                    dv3_mw_by_occ_names[occupation][race] = curr_names
                if occupation in dv3_mw_occ[race]:
                    dv3_mw_occ[race][occupation].append(new_outs)
                    dv3_mw_occ_names[race][occupation].append(curr_names)
                else:
                    dv3_mw_occ[race][occupation] = new_outs
                    dv3_mw_occ_names[race][occupation] = curr_names
            else:
                dv3_mw[race] = [[new_outs]]
                dv3_mw_names[race] = [[curr_names]]
                dv3_mw_by_occ[occupation][race] = new_outs
                dv3_mw_by_occ_names[occupation][race] = curr_names
                dv3_mw_occ[race] = {occupation: new_outs}
                dv3_mw_occ_names[race] = {occupation: curr_names}
        temps = []
        temps_names = []
        for race in df[gender_col].unique():        
            if og_computation:
                temp = pprint(get_marked_words_og(df, ['M'], [gender_col],[race], occupation))
            else:
                temp = pprint(get_marked_words_occ(df, ['M'], [gender_col],[race], occupation))
            new_temp = list()
            curr_names = list()
            for word in temp:
                if remove_names and (word in lower_names or word[:-1] in lower_names):
                    curr_names.append(word)
                else:
                    new_temp.append(word)
            temps.extend(new_temp)
            temps_names.extend(curr_names)

        seen = Counter(temps).most_common()
        seen_names = Counter(temps_names).most_common()
        num_seen = len(df[gender_col].unique()) - 1
        m_words = [w for w, c in seen if c == num_seen]
        m_words_names = [w for w, c in seen_names if c == num_seen]
        if 'M' in dv3_mw:
            dv3_mw['M'].append(m_words)
            dv3_mw_names['M'].append(m_words_names)
        else:
            dv3_mw['M'] = m_words
            dv3_mw_names['M'] = m_words_names
        dv3_mw_by_occ[occupation]['M'] = m_words
        dv3_mw_by_occ_names[occupation]['M'] = m_words_names

    return dv3_mw, dv3_mw_counts, dv3_mw_names, dv3_mw_by_occ, dv3_mw_by_occ_counts, dv3_mw_by_occ_names, dv3_mw_occ, dv3_mw_occ_counts, dv3_mw_occ_names

In [41]:
dv3_mw_gender_og, dv3_mw_counts_gender_og, dv3_mw_names_gender_og, dv3_mw_by_occ_gender_og, dv3_mw_by_occ_counts_gender_og, dv3_mw_by_occ_names_gender_og, dv3_mw_occ_gender_og, dv3_mw_occ_counts_gender_og, dv3_mw_occ_names_gender_og = compute_marked_word_og_and_calibrated(df_gender[df_gender['occupation'] == 'software engineer'], og_computation=True)

dv3_mw_gender_cal, dv3_mw_counts_gender_cal, dv3_mw_names_gender_cal, dv3_mw_by_occ_gender_cal, dv3_mw_by_occ_counts_gender_cal, dv3_mw_by_occ_names_gender_cal, dv3_mw_occ_gender_cal, dv3_mw_occ_counts_gender_cal, dv3_mw_occ_names_gender_cal = compute_marked_word_og_and_calibrated(df_gender[df_gender['occupation'] == 'software engineer'], og_computation=False)

his, -27.385
he, -24.110
him, -10.346
david, -10.068
james, -6.627
alex, -6.531
michael, -5.932
daniel, -5.879
himself, -5.144
jonathan, -5.021
her, 31.804
she, 23.643
maya, 16.410
women, 13.315
dr, 8.446
sarah, 7.884
girls, 6.113
female, 5.877
stem, 5.291
in, 5.136
and, 0.000
in, 0.000
a, 0.000
, 0.000
to, 0.000
the, 0.000
of, 0.000
for, 0.000
tech, 0.000
her, 0.000
and, 0.000
in, 0.000
a, 0.000
, 0.000
to, 0.000
the, 0.000
of, 0.000
for, 0.000
tech, 0.000
her, 0.000
his, -27.546
he, -24.252
him, -10.441
david, -10.128
james, -6.666
thompson, -6.569
michael, -5.967
daniel, -5.914
himself, -5.174
jonathan, -5.051
their, 23.963
they, 21.101
alex, 13.716
nonbinary, 13.045
identity, 11.020
morgan, 9.003
inclusivity, 8.956
lgbtq, 8.684
inclusive, 7.740
are, 7.537
her, -31.804
she, -23.643
maya, -16.410
women, -13.315
dr, -8.446
sarah, -7.884
girls, -6.113
female, -5.877
stem, -5.291
in, -5.136
his, 27.385
he, 24.110
him, 10.346
david, 10.068
james, 6.627
alex, 6.531
michael, 5.932
daniel, 

In [42]:
not_in_calibrated = list()
not_in_og = list()
not_in_calibrated_by_gender = {'M': [], 'F': [], 'N': []}
not_in_og_by_gender = {'M': [], 'F': [], 'N': []}
for g in ['M', 'F', 'N']:
    str_to_print = f'{g} & '
    for word in dv3_mw_by_occ_gender_og['software engineer'][g]:
        if word not in dv3_mw_by_occ_gender_cal['software engineer'][g]:
            not_in_calibrated.append(word)
            not_in_calibrated_by_gender[g].append(word)
    for word in dv3_mw_by_occ_gender_cal['software engineer'][g]:
        if word not in dv3_mw_by_occ_gender_og['software engineer'][g]:
            not_in_og.append(word)
            not_in_og_by_gender[g].append(word)

print('not in calibrated')
print(not_in_calibrated)

print('not in og')
print(not_in_og)

# Marked Words displayed are the words identified by Cheng et al.'s Marked Words method and not by our Calibrated Marked Words method.
print('Gender & Marked Words\\\\')
print('\\hline')
for g in ['M', 'F', 'N']:
    str_to_print = ''
    for word in not_in_calibrated_by_gender[g]:
        str_to_print += f'{word}, '
    print(f'{g} & {str_to_print[:-2]}')

# Calibrated Marked Words displayed are the words identified by our Calibrated Marked Words method and not by Cheng et al.'s Marked Words method.
print('Gender & Calibrated Marked Words \\\\')
print('\\hline')
for g in ['M', 'F', 'N']:
    str_to_print = ''
    for word in not_in_og_by_gender[g]:
        str_to_print += f'{word}, '
    print(f'{g} & {str_to_print[:-2]}\\\\')


not in calibrated
['back', 'boy', 'children', 'one', 'being', 'i', 'been', 'we', 'state', 'were', 'value', 'be', 'feel', 'felt', 'should', 'our', 'expression']
not in og
['projects', 'github', 'online', 'values', 'keen', 'technologies', 'lifestyle', 'detailoriented', 'enjoys', 'analytical', 'underprivileged', 'struggles', 'collaborative', 'honed', 'burgeoning', 'boundaries', 'management', 'innovatech', 'cycling', 'carter', 'spends', 'startup', 'kubernetes', 'streamlined', 'knack', 'outdoor', 'contributes', 'repositories', 'avid', 'attracting', 'streamline', 'max', 'clean', 'aspirations', 'peers', 'frameworks', 'tackling', 'finds', 'manageable', 'problems', 'clients', 'mobile', 'fitness', 'jason', 'designer', 'reviews', 'immersed', 'best', 'adaptable', 'interned', 'stay', 'healthy', 'courses', 'pays', 'hours', 'andrews', 'tools', 'enthusiast', 'inc', 'graduating', 'takes', 'years', 'regularly', 'propelled', 'methodical', 'prominence', 'reynolds', 'marked', 'jameson', 'blockchain', 'jona